In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import sklearn
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasRegressor

df = pd.read_csv('../input/np-regression-tables/scores1.csv')

In [ ]:
df.describe()

In [ ]:
table = df.to_numpy()

table[:,-1] /= table[:,-2]

X, y = table[:,:-1], table[:,-1]

In [ ]:
#X = X[:, [0, 1, 2, 3, 4, 9]]

In [ ]:
def get_NN():
    clf = Sequential()
    clf.add(Dense(1, activation='sigmoid', input_dim=X.shape[1]))
    #clf.add(Dense(9, activation='relu'))
    #clf.add(Dense(9, activation='relu'))
    #clf.add(Dense(9, activation='relu'))
    #clf.add(Dense(1, activation='sigmoid'))
    clf.compile(loss='mse')
    return clf


# clf = KerasClassifier(TwoLayerFeedForward(), epochs=100, batch_size=500, verbose=0)
NN = KerasRegressor(get_NN, epochs=100, batch_size=64, verbose=0)

pipe = Pipeline([
    ('scaler', StandardScaler()), 
    #('PCA', PCA()),
    ('NN', NN)])

In [ ]:
trans = KFold(n_splits=20, shuffle=True, random_state=9)

scores_train = []
scores_valid = []

for train_idx, test_idx in trans.split(X, y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[test_idx], y[test_idx]
    
    pipe.fit(X_train, y_train)
    
    t, v = pipe.score(X_train, y_train), pipe.score(X_valid, y_valid)
    
    print(f"train score = {t} \t validation score = {v}")
    
    scores_train.append(t)
    scores_valid.append(v)

np.mean(scores_train), np.mean(scores_valid)

In [ ]:
pipe.fit(X, y)

In [ ]:
scaler = pipe.steps[0][1]

NN = pipe.steps[1][1]

In [ ]:
weights = sum([list(layer.get_weights()) for layer in NN.model.layers], [])

In [ ]:
!pip install cost2fitness

In [ ]:
from cost2fitness import NNStep, Pl, Relu, Sigmoid


fastNN = Pl([
    NNStep(from_size = 10, to_size = 9),
    Relu(),
    NNStep(from_size = 9, to_size = 9),
    Relu(),
    NNStep(from_size = 9, to_size = 9),
    Relu(),
    NNStep(from_size = 9, to_size = 9),
    Relu(),
    NNStep(from_size = 9, to_size = 1),
    Sigmoid()
])

fastNN.set_weights(weights)

y0 = pipe.predict(X)

y1 = np.array([fastNN.transform(x)[0] for x in scaler.transform(X)])


np.sum((y0-y1)**2)


In [ ]:
df2 = pd.read_csv('../input/np-regression-tables/scores2.csv').to_numpy()

df2[:,-1] /= df2[:,-2]

In [ ]:
X_test, y_test = df2[:,:-1], df2[:, -1]

In [ ]:
y0 = np.array([fastNN.transform(x)[0] for x in scaler.transform(X_test)])

np.mean((y0-y_test)**2)

In [ ]:
y_test

In [ ]:
y0

In [ ]:
pipe.fit(np.vstack((X,X_test)), np.concatenate([y, y_test]))

In [ ]:
df3 = pd.read_csv('../input/np-regression-tables/scores3.csv').to_numpy()

df3[:,-1] /= df3[:,-2]

In [ ]:
y0 = pipe.predict(df3[:,:-1])

np.mean((y0-df3[:,-1])**2)